<a href="https://colab.research.google.com/github/SakshiGehani/J015_NLPassignments/blob/ClassWork/CW8_SklearnPipeline_J015_4_2_20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

from google.colab import files
uploaded = files.upload()

Saving Womens Clothing E-Commerce Reviews.csv to Womens Clothing E-Commerce Reviews.csv


In [0]:
import io
df = pd.read_csv(io.BytesIO(uploaded['Womens Clothing E-Commerce Reviews.csv']))

In [0]:
df.head()

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [0]:
#df.columns
df = df.drop(["Unnamed: 0"], 1)

In [0]:
df.head()     ## possible target variables: Recommended(Binary classi.), rating(Multivariate classification)-- in that case remove 'recommended', coz it is a derived feature=>model becomes biased


,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [0]:
# Concating title and rev text
#df['Review'] = df[['Title', 'Review Text']].apply(lambda x: ''.join(x), axis=1)
rev = df['Title'].astype(str) + " " + df['Review Text']
df.insert(4, "Review", rev)

In [0]:
df.Review = df.Review.astype(str)

In [0]:
df.head()

,Clothing ID,Age,Title,Review Text,Review,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,nan Absolutely wonderful - silky and sexy and ...,4,1,0,Initmates,Intimate,Intimates
1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,nan Love this dress! it's sooo pretty. i hap...,5,1,4,General,Dresses,Dresses
2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,Some major design flaws I had such high hopes ...,3,0,0,General,Dresses,Dresses
3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...","My favorite buy! I love, love, love this jumps...",5,1,0,General Petite,Bottoms,Pants
4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,Flattering shirt This shirt is very flattering...,5,1,6,General,Tops,Blouses


In [0]:
#df.Review[0]

In [0]:
# One hot encode-Divisionname, Dept name, Class name-- not label encode; we do not want categories to be ranked
'''df["Division Name"] = pd.get_dummies(df["Division Name"], prefix_sep='_', drop_first=True)
df["Department Name"] = pd.get_dummies(df["Department Name"], prefix_sep='_', drop_first=True)
df["Class Name"] = pd.get_dummies(df["Class Name"], prefix_sep='_', drop_first=True)'''
new_df = df.iloc[:, 8:11]

new_df = pd.get_dummies(new_df, drop_first=True)
new_df.head()

,Division Name_General Petite,Division Name_Initmates,Department Name_Dresses,Department Name_Intimate,Department Name_Jackets,Department Name_Tops,Department Name_Trend,Class Name_Casual bottoms,Class Name_Chemises,Class Name_Dresses,Class Name_Fine gauge,Class Name_Intimates,Class Name_Jackets,Class Name_Jeans,Class Name_Knits,Class Name_Layering,Class Name_Legwear,Class Name_Lounge,Class Name_Outerwear,Class Name_Pants,Class Name_Shorts,Class Name_Skirts,Class Name_Sleep,Class Name_Sweaters,Class Name_Swim,Class Name_Trend
0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
df = df.drop(["Division Name", "Department Name", "Class Name"], 1)
df = pd.concat([df, new_df], axis=1)

In [0]:
df.head()

,Clothing ID,Age,Title,Review Text,Review,Rating,Recommended IND,Positive Feedback Count,Division Name_General Petite,Division Name_Initmates,Department Name_Dresses,Department Name_Intimate,Department Name_Jackets,Department Name_Tops,Department Name_Trend,Class Name_Casual bottoms,Class Name_Chemises,Class Name_Dresses,Class Name_Fine gauge,Class Name_Intimates,Class Name_Jackets,Class Name_Jeans,Class Name_Knits,Class Name_Layering,Class Name_Legwear,Class Name_Lounge,Class Name_Outerwear,Class Name_Pants,Class Name_Shorts,Class Name_Skirts,Class Name_Sleep,Class Name_Sweaters,Class Name_Swim,Class Name_Trend
0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,nan Absolutely wonderful - silky and sexy and ...,4,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,nan Love this dress! it's sooo pretty. i hap...,5,1,4,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,Some major design flaws I had such high hopes ...,3,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...","My favorite buy! I love, love, love this jumps...",5,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,Flattering shirt This shirt is very flattering...,5,1,6,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
df.isna()

,Clothing ID,Age,Title,Review Text,Review,Rating,Recommended IND,Positive Feedback Count,Division Name_General Petite,Division Name_Initmates,Department Name_Dresses,Department Name_Intimate,Department Name_Jackets,Department Name_Tops,Department Name_Trend,Class Name_Casual bottoms,Class Name_Chemises,Class Name_Dresses,Class Name_Fine gauge,Class Name_Intimates,Class Name_Jackets,Class Name_Jeans,Class Name_Knits,Class Name_Layering,Class Name_Legwear,Class Name_Lounge,Class Name_Outerwear,Class Name_Pants,Class Name_Shorts,Class Name_Skirts,Class Name_Sleep,Class Name_Sweaters,Class Name_Swim,Class Name_Trend
0,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23481,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
23482,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
23483,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
23484,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [0]:
#fill na with space

In [0]:
df = df.drop(["Title", "Review Text"], 1)

In [0]:
# importing required modules
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [0]:
## Preprocessing data

stop_words = set(stopwords.words('english'))      # for unique entries -> set()
stop_words.add("nan")
words = set(nltk.corpus.words.words())
def preprocess(sen):
  sen = sen.lower()
  no_punc = sen.translate(sen.maketrans('', '', '!"$%&\'()#@*+,-./:;<=>?[\\]^_`{|}~'))      ## removing all punctuation marks
  filtered = no_punc.translate(no_punc.maketrans('', '', '#@0123456789'))           ## removing digits
  tokens = word_tokenize(filtered)                        ## creating a list of tokens
  no_stop_words = [w for w in tokens if not w in stop_words]     # eliminate stop words;  returns list of lists
  sent = " ".join(w for w in no_stop_words)
  sent = " ".join(w for w in nltk.wordpunct_tokenize(sent) \
            if w.lower() in words or not w.isalpha())                ## eliminating all non-english words
  
  return sent

In [0]:
cleaned = df.Review.apply(preprocess)

In [0]:
cleaned

0              absolutely wonderful silky sexy comfortable
1        love dress pretty find store glad never would ...
2        major design high dress really work initially ...
3        favorite buy love love love fun flirty fabulou...
4        flattering shirt shirt flattering due adjustab...
                               ...                        
23481    great dress many happy snag dress great price ...
23482    wish made cotton maternity clothes soft stretc...
23483    cute see fit well top see never would worked g...
23484    cute dress perfect summer bought dress wedding...
23485    please make like one dress lovely platinum fem...
Name: Review, Length: 23486, dtype: object

In [0]:
df.head()

,Clothing ID,Age,Review,Rating,Recommended IND,Positive Feedback Count,Division Name_General Petite,Division Name_Initmates,Department Name_Dresses,Department Name_Intimate,Department Name_Jackets,Department Name_Tops,Department Name_Trend,Class Name_Casual bottoms,Class Name_Chemises,Class Name_Dresses,Class Name_Fine gauge,Class Name_Intimates,Class Name_Jackets,Class Name_Jeans,Class Name_Knits,Class Name_Layering,Class Name_Legwear,Class Name_Lounge,Class Name_Outerwear,Class Name_Pants,Class Name_Shorts,Class Name_Skirts,Class Name_Sleep,Class Name_Sweaters,Class Name_Swim,Class Name_Trend
0,767,33,nan Absolutely wonderful - silky and sexy and ...,4,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1080,34,nan Love this dress! it's sooo pretty. i hap...,5,1,4,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1077,60,Some major design flaws I had such high hopes ...,3,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1049,50,"My favorite buy! I love, love, love this jumps...",5,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,847,47,Flattering shirt This shirt is very flattering...,5,1,6,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(cleaned)

In [0]:
print(vectorizer.get_feature_names())

['aa', 'abbey', 'abdomen', 'abdominal', 'abhor', 'ability', 'abject', 'able', 'abnormal', 'abnormally', 'abroad', 'abruptly', 'absence', 'absolute', 'absolutely', 'absorbent', 'abstract', 'absurd', 'absurdly', 'abundance', 'abundant', 'abut', 'accent', 'accentuate', 'accept', 'acceptable', 'acceptably', 'accepted', 'access', 'accessory', 'accident', 'accidental', 'accidentally', 'accommodate', 'accommodating', 'accompany', 'accomplish', 'accomplishment', 'accord', 'accordance', 'according', 'accordingly', 'accordion', 'account', 'accumulate', 'accuracy', 'accurate', 'accurately', 'ace', 'acetate', 'achieve', 'acid', 'acknowledge', 'acquaintance', 'acquisition', 'across', 'acrylic', 'act', 'action', 'active', 'actively', 'activity', 'actual', 'actuality', 'actually', 'ad', 'add', 'added', 'addict', 'addicted', 'addiction', 'addition', 'additional', 'additionally', 'address', 'adequate', 'adequately', 'adhesive', 'adieu', 'adjacent', 'adjust', 'adjustable', 'adjustment', 'admirable', 'ad

In [0]:
print(X.toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [0]:
X = pd.DataFrame(X.toarray(), columns=(vectorizer.get_feature_names()))

In [0]:
from sklearn.model_selection import train_test_split as tts
X_train, X_test, y_train, y_test = tts(X, df["Recommended IND"], random_state = 42, test_size = 0.3)

In [0]:
!pip install xgboost

In [0]:
# applying base model XGBoost on the data returned by the tf/idf vectorizer

from xgboost import XGBClassifier
from sklearn.metrics import classification_report

# fit model on training data
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
# make predictions for test data
xg_pred = xgb.predict(X_test)

In [0]:
xg_probs = xgb.predict_proba(X_test)
print("\n Classification Report for test data : \n", classification_report(xg_pred, y_test))


 Classification Report for test data : 
               precision    recall  f1-score   support

           0       0.28      0.82      0.42       425
           1       0.99      0.86      0.92      6621

    accuracy                           0.86      7046
   macro avg       0.63      0.84      0.67      7046
weighted avg       0.94      0.86      0.89      7046



In [0]:
xg_probs1 = xgb.predict_proba(X_train)
print("\n Classification Report for train data : \n", classification_report(xgb.predict(X_train), y_train))


 Classification Report for train data : 
               precision    recall  f1-score   support

           0       0.32      0.89      0.47      1053
           1       0.99      0.87      0.93     15387

    accuracy                           0.87     16440
   macro avg       0.66      0.88      0.70     16440
weighted avg       0.95      0.87      0.90     16440



## 11-02-2020 
## Function Transform, MinMax Scaler

In [0]:
import numpy as np

In [0]:
def minMaxScaler(X):
  for i in range (0, X.shape[1]):
    a = X[:, i]   
    X[:, i] = (a - a.min()) / (a.max() - a.min())
  return X

In [0]:
df0 = np.array([[0.0, 10.0], [0.13216, 12.11837], [0.25379, 42.05027], [0.30874, 13.11784]])

In [0]:
df0

array([[ 0.     , 10.     ],
       [ 0.13216, 12.11837],
       [ 0.25379, 42.05027],
       [ 0.30874, 13.11784]])

In [0]:
from sklearn.preprocessing import FunctionTransformer
transformer = FunctionTransformer(minMaxScaler)
#X = np.array([[0, 1], [2, 3]])
#transformer.transform(df0)
transformer.fit_transform(df0)

array([[0.        , 0.        ],
       [0.42806245, 0.06609523],
       [0.82201853, 1.        ],
       [1.        , 0.09727968]])

In [0]:
transformer.inverse_transform(df0)

array([[0.        , 0.        ],
       [0.42806245, 0.06609523],
       [0.82201853, 1.        ],
       [1.        , 0.09727968]])

In [0]:
df0

array([[0.        , 0.        ],
       [0.42806245, 0.06609523],
       [0.82201853, 1.        ],
       [1.        , 0.09727968]])

## Credit card default dataset

In [0]:
df2 = 

In [0]:
from sklearn.base import BaseEstimator, TransformerMixin


In [0]:
class PositionalSelector(BaseEstimator, TransformerMixin):
  def __init__(self, positions):
    self.positions = positions

  def fit(self,X,y=None):
    return self

  def transform(self,X):
    return np.array(X)[:self.positions]

  def inverse_transform(X):
    return X

In [0]:
class Min_Max_Scaler(BaseEstimator, TransformerMixin):
  def __init__(self):

  def fit(self,X,y=None):
    return self

  def transform(self,X):
    return transformer.transform(self)
    

In [0]:
def create_pipeline():
  # Categorical -> One hot encoding
  # Numerical -> Scaling
  numerical_indices = [0,4,12]
  categorical_indices = [1,5,7]

  p1 = make_pipeline(np.PositionalSelector(categorical_indices)''', np.StripString()''', np.SimpleOneHotEncoder())
  p2 = make_pipeline(np.PositionalSelector(categorical_indices), StandardScaler())

  feats = FeatureUnion([
                        ('numericals', p1),
                        ('categoricals', p2)
  ])

  pipeline = Pipeline([
                       ('pre', feats),
                       ('estimator',GradientBoostingClassifier(max_depth = 4, n_estimator = 100))
  ])
  return pipeline